In [1]:
%tensorflow_version
#!pip install q keras==2.6.0
#!pip install q keras==2.0.5

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import typing
from typing import Any, Tuple

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
num_samples = 10000 ## limitation because of hardware constrainsts
max_len = 200 ## char limitation for sentences

In [5]:
#"Turkish Noise Data/tr.train"
noisy_train = []
noisy_chars = set()
normal_train = []
normal_chars = set()
with open('/content/drive/My Drive/Turkish_Noise_Data/tr.train', "r", encoding="utf-8") as f:    ##chek if you need eos and sos for both inp and tgt
    lines = f.read().split("\n") 
    for line in lines[: min(num_samples, len(lines) - 1)]:
        #sentence = "\t" + line + "\n"  ## \t is the start char and \n is the end char
        sentence = line
        if len(sentence) <= max_len:
          normal_train.append(sentence)
        for char in sentence:
          if char not in normal_chars:
            normal_chars.add(char)

with open('/content/drive/My Drive/Turkish_Noise_Data/tr_noisy.train', "r", encoding="utf-8") as f:  
    lines = f.read().split("\n") 
    for line in lines[: min(num_samples, len(lines) - 1)]:
        #sentence = "\t" + line + "\n"  ## \t is the start char and \n is the end char
        sentence = line
        if len(sentence) <= max_len:
          noisy_train.append(sentence)
        for char in sentence:
          if char not in noisy_chars:
            noisy_chars.add(char)


print(noisy_chars) ###

{'/', 'ğ', 'h', '@', '9', '[', 'ı', "'", '$', '`', 'ñ', 'ü', '2', 'a', ',', 's', '”', '©', 'x', 't', '!', 'm', 'w', '1', 'l', '&', 'u', 'k', ' ', 'á', 'd', 'z', 'b', 'é', '%', 'y', 'j', 'v', '(', 'c', '3', '♫', 'û', '#', 'î', 'o', '0', '7', 'f', 'n', '-', 'ç', 'q', 'í', ']', '4', '8', ';', '5', '’', ')', 'i', 'r', 'ö', 'ş', '.', '~', 'ó', '“', 'p', 'â', '?', 'g', '6', ':', 'e', '—'}


In [6]:
print(noisy_train[0])

biz diplomatlar , tejletler arasındaki natışmjlar ve sorunları çöömek için eğitildiç .


In [7]:
noisy_chars = (list(noisy_chars))
normal_chars = (list(normal_chars))
num_encoder_tokens = len(noisy_chars)
num_decoder_tokens = len(normal_chars)
max_encoder_seq_length = max([len(txt) for txt in noisy_train])
max_decoder_seq_length = max([len(txt) for txt in normal_train])

input_token_index = dict([(char, i) for i, char in enumerate(noisy_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(normal_chars)])

input_token_index_rev = dict([(i, char) for i, char in enumerate(noisy_chars)])
target_token_index_rev = dict([(i, char) for i, char in enumerate(normal_chars)])

In [8]:
print(input_token_index)

{'/': 0, 'ğ': 1, 'h': 2, '@': 3, '9': 4, '[': 5, 'ı': 6, "'": 7, '$': 8, '`': 9, 'ñ': 10, 'ü': 11, '2': 12, 'a': 13, ',': 14, 's': 15, '”': 16, '©': 17, 'x': 18, 't': 19, '!': 20, 'm': 21, 'w': 22, '1': 23, 'l': 24, '&': 25, 'u': 26, 'k': 27, ' ': 28, 'á': 29, 'd': 30, 'z': 31, 'b': 32, 'é': 33, '%': 34, 'y': 35, 'j': 36, 'v': 37, '(': 38, 'c': 39, '3': 40, '♫': 41, 'û': 42, '#': 43, 'î': 44, 'o': 45, '0': 46, '7': 47, 'f': 48, 'n': 49, '-': 50, 'ç': 51, 'q': 52, 'í': 53, ']': 54, '4': 55, '8': 56, ';': 57, '5': 58, '’': 59, ')': 60, 'i': 61, 'r': 62, 'ö': 63, 'ş': 64, '.': 65, '~': 66, 'ó': 67, '“': 68, 'p': 69, 'â': 70, '?': 71, 'g': 72, '6': 73, ':': 74, 'e': 75, '—': 76}


In [9]:
print(len(noisy_train))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

9118
Number of unique input tokens: 77
Number of unique output tokens: 77
Max sequence length for inputs: 200
Max sequence length for outputs: 200


In [10]:
batch_size = 64  # Batch size for training. ##was 16
epochs = 100  # Number of epochs to train for.
num_units = 128  # Latent dimensionality
dropout = 0.2

In [11]:
def tokenize(sent,index_dict):
  arr = []
  for char in sent:
    arr.append(index_dict[char])
  return arr

def toText(arr,index_dict):
  sent = ""
  for token in arr:
    sent += index_dict[token]
  return sent

In [12]:
# for i,line in enumerate(noisy_train):
#   noisy_train[i] = tokenize(line,input_token_index)
#   while len(noisy_train[i]) < max_encoder_seq_length:
#     noisy_train[i].append(input_token_index['\n'])

# for i,line in enumerate(normal_train):
#   normal_train[i] = tokenize(line,target_token_index)
#   while len(normal_train[i]) < max_decoder_seq_length:
#     normal_train[i].append(target_token_index['\n'])

In [13]:
# print(noisy_train[0])
# print(toText(noisy_train[0],input_token_index_rev))

In [14]:
def char_std(text):
  # Split accecented characters.
  #text = tf_text.normalize_utf8(text, 'NFKD')
  #text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  #text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  #text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['\t', text, '\n'], separator='')
  return text

In [15]:
max_vocab_size = 5000

input_text_processor = tf.keras.layers.TextVectorization(
    standardize=char_std,
    split = 'character',
    max_tokens=max_vocab_size)

input_text_processor.adapt(noisy_train)
input_text_processor.get_vocabulary()  ## test vocab

output_text_processor  = tf.keras.layers.TextVectorization(
    standardize=char_std,
    split = 'character',
    max_tokens=max_vocab_size)

output_text_processor.adapt(normal_train)

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((noisy_train,normal_train)).shuffle(len(noisy_train))
dataset = dataset.batch(batch_size)

#example_input_batch, example_target_batch =  dataset.take(1)

for example_input_batch, example_target_batch in dataset.take(2):
  example_input_batch = example_input_batch
  example_target_batch = example_target_batch
  print(example_input_batch[0])
  example_tokens = input_text_processor(example_input_batch)
  print(example_tokens[0])
  input_vocab = np.array(input_text_processor.get_vocabulary())
  tokens = input_vocab[example_tokens[0].numpy()]
  print(''.join(tokens))
  print(example_target_batch[0])
  break

tf.Tensor(b'ge\xc3\xa7dnlerde \xc3\xb6\xc4\x9frmndi\xc3\xb6 ; minnettarz\xc4\xb1k ve olumlava negatif etkile\xc5\x9fimleri dengeliyormu\xc5\x9f .', shape=(), dtype=string)
tf.Tensor(
[24 25  4 26 11  7  8  4  6 11  4  2 33 30  6 12  7 11  5 33  2 51  2 12
  5  7  7  4 16 16  3  6 19 10  9  2 27  4  2 17  8 14 12  8  3 27  3  2
  7  4 25  3 16  5 35  2  4 16  9  5  8  4 22  5 12  8  4  6  5  2 11  4
  7 25  4  8  5 13 17  6 12 14 22  2 21 23  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], shape=(190,), dtype=int64)
	geçdnlerde öğrmndiö ; minnettarzık ve olumlava negatif etkileşimleri dengeliyormuş .

tf.Tensor(b'ge\xc3\xa7enlerde \xc3\xb6\xc4\x9frendim ; minnettarl\xc4\xb1k ve olumlama negatif etkile\xc5\x9fimleri dengeli

In [17]:
embedding_dim = 1
units = 128 ##latent dim was 1024

In [18]:
# Encoder
class Encoder(tf.keras.layers.Layer):
  def __init__(self, input_vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.input_vocab_size = input_vocab_size

    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.input_vocab_size,
                                               embedding_dim)

    # The GRU RNN layer processes those vectors sequentially.
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   # Return the sequence and state
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',dropout=0.2)

  def call(self, tokens, state=None):

    # 2. The embedding layer looks up the embedding for each token.
    vectors = self.embedding(tokens)

    # 3. The GRU processes the embedding sequence.
    #    output shape: (batch, s, enc_units)
    #    state shape: (batch, enc_units)
    output, state = self.gru(vectors, initial_state=state)
    # 4. Returns the new sequence and its state.
    return output, state

In [19]:
encoder = Encoder(input_text_processor.vocabulary_size(),
                  embedding_dim, units)

In [20]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    # For Eqn. (4), the  Bahdanau attention
    self.W1 = tf.keras.layers.Dense(units, use_bias=False)
    self.W2 = tf.keras.layers.Dense(units, use_bias=False)

    self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, value, mask):

    # From Eqn. (4), `W1@ht`.
    w1_query = self.W1(query)

    # From Eqn. (4), `W2@hs`.
    w2_key = self.W2(value)

    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
    value_mask = mask

    context_vector, attention_weights = self.attention(
        inputs = [w1_query, value, w2_key],
        mask=[query_mask, value_mask],
        return_attention_scores = True,
    )

    return context_vector, attention_weights

In [21]:
attention_layer = BahdanauAttention(units)

In [22]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, output_vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.dec_units = dec_units
    self.output_vocab_size = output_vocab_size
    self.embedding_dim = embedding_dim

    # For Step 1. The embedding layer convets token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.output_vocab_size,
                                               embedding_dim)

    # For Step 2. The RNN keeps track of what's been generated so far.
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   # Return the sequence and state
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',dropout=0.2)

    # For step 3. The RNN output will be the query for the attention layer.
    self.attention = BahdanauAttention(self.dec_units)

    # For step 4. Eqn. (3): converting `ct` to `at`
    self.Wc = tf.keras.layers.Dense(dec_units, activation=tf.math.tanh,
                                    use_bias=False)

    # For step 5. This fully connected layer produces the logits for each
    # output token.
    self.fc = tf.keras.layers.Dense(self.output_vocab_size)

In [23]:
class DecoderInput(typing.NamedTuple):
  new_tokens: Any
  enc_output: Any
  mask: Any

class DecoderOutput(typing.NamedTuple):
  logits: Any
  attention_weights: Any

In [24]:
def call(self,
         inputs: DecoderInput,
         state=None) -> Tuple[DecoderOutput, tf.Tensor]:

  # if state is not None:
  #   shape_checker(state, ('batch', 'dec_units'))

  # Step 1. Lookup the embeddings
  vectors = self.embedding(inputs.new_tokens)

  # Step 2. Process one step with the RNN
  rnn_output, state = self.gru(vectors, initial_state=state)

  # Step 3. Use the RNN output as the query for the attention over the
  # encoder output.
  context_vector, attention_weights = self.attention(
      query=rnn_output, value=inputs.enc_output, mask=inputs.mask)

  # Step 4. Eqn. (3): Join the context_vector and rnn_output
  #     [ct; ht] shape: (batch t, value_units + query_units)
  context_and_rnn_output = tf.concat([context_vector, rnn_output], axis=-1)

  # Step 4. Eqn. (3): `at = tanh(Wc@[ct; ht])`
  attention_vector = self.Wc(context_and_rnn_output)

  # Step 5. Generate logit predictions:
  logits = self.fc(attention_vector)

  return DecoderOutput(logits, attention_weights), state

In [25]:
Decoder.call = call

In [26]:
decoder = Decoder(output_text_processor.vocabulary_size(),
                  embedding_dim, units)

In [27]:
class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'masked_loss'
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

  def __call__(self, y_true, y_pred):

    # Calculate the loss for each item in the batch.
    loss = self.loss(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, tf.float32)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)

In [28]:
class TrainModel(tf.keras.Model):
  def __init__(self, embedding_dim, units,
               input_text_processor,
               output_text_processor, 
               use_tf_function=True):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(input_text_processor.vocabulary_size(),
                      embedding_dim, units)
    decoder = Decoder(output_text_processor.vocabulary_size(),
                      embedding_dim, units)

    self.encoder = encoder
    self.decoder = decoder
    self.input_text_processor = input_text_processor
    self.output_text_processor = output_text_processor
    self.use_tf_function = use_tf_function

  def train_step(self, inputs):
    if self.use_tf_function:
      return self._tf_train_step(inputs)
    else:
      return self._train_step(inputs)

In [29]:
def _preprocess(self, input_text, target_text):

  # Convert the text to token IDs
  input_tokens = self.input_text_processor(input_text)
  target_tokens = self.output_text_processor(target_text)

  # Convert IDs to masks.
  input_mask = input_tokens != 0

  target_mask = target_tokens != 0

  return input_tokens, input_mask, target_tokens, target_mask

In [30]:
TrainModel._preprocess = _preprocess

In [31]:
def _train_step(self, inputs):
  input_text, target_text = inputs   ## change here

  (input_tokens, input_mask,
   target_tokens, target_mask) = self._preprocess(input_text, target_text)

  max_target_length = tf.shape(target_tokens)[1]

  with tf.GradientTape() as tape:
    # Encode the input
    enc_output, enc_state = self.encoder(input_tokens)

    # Initialize the decoder's state to the encoder's final state.
    # This only works if the encoder and decoder have the same number of
    # units.
    dec_state = enc_state
    loss = tf.constant(0.0)

    for t in tf.range(max_target_length-1):
      # Pass in two tokens from the target sequence:
      # 1. The current input to the decoder.
      # 2. The target for the decoder's next prediction.
      new_tokens = target_tokens[:, t:t+2]
      step_loss, dec_state = self._loop_step(new_tokens, input_mask,
                                             enc_output, dec_state)
      loss = loss + step_loss

    # Average the loss over all non padding tokens.
    average_loss = loss / tf.reduce_sum(tf.cast(target_mask, tf.float32))

  # Apply an optimization step
  variables = self.trainable_variables 
  gradients = tape.gradient(average_loss, variables)
  self.optimizer.apply_gradients(zip(gradients, variables))

  # Return a dict mapping metric names to current value
  return {'batch_loss': average_loss}

In [32]:
TrainModel._train_step = _train_step

In [33]:
def _loop_step(self, new_tokens, input_mask, enc_output, dec_state):
  input_token, target_token = new_tokens[:, 0:1], new_tokens[:, 1:2]

  # Run the decoder one step.
  decoder_input = DecoderInput(new_tokens=input_token,
                               enc_output=enc_output,
                               mask=input_mask)

  dec_result, dec_state = self.decoder(decoder_input, state=dec_state)

  # `self.loss` returns the total for non-padded tokens
  y = target_token
  y_pred = dec_result.logits
  step_loss = self.loss(y, y_pred)

  return step_loss, dec_state

In [34]:
TrainModel._loop_step = _loop_step

In [35]:
charModel = TrainModel(
    embedding_dim, units,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor,
    use_tf_function=False)

# Configure the loss and optimizer
charModel.compile(
    optimizer=tf.optimizers.Adam(),
    loss=MaskedLoss(),
)

In [36]:
## loss should start near this
np.log(output_text_processor.vocabulary_size())

4.3694478524670215

In [37]:
%%time
for n in range(10):
  print(charModel.train_step([example_input_batch, example_target_batch]))
print()

{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.31936>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.3121853>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.302684>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.2896023>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.271751>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.2476535>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.2153234>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.172043>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.11411>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=4.036419>}

CPU times: user 37.7 s, sys: 1.01 s, total: 38.7 s
Wall time: 42.2 s


In [38]:
@tf.function(input_signature=[[tf.TensorSpec(dtype=tf.string, shape=[None]),
                               tf.TensorSpec(dtype=tf.string, shape=[None])]])
def _tf_train_step(self, inputs):
  return self._train_step(inputs)

In [39]:
TrainModel._tf_train_step = _tf_train_step

In [40]:
charModel.use_tf_function = True

In [41]:
charModel.train_step([example_input_batch, example_target_batch])

{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.932152>}

In [42]:
%%time
for n in range(10):
  print(charModel.train_step([example_input_batch, example_target_batch]))
print()

{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.7933333>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.617003>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.4459004>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.5871983>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.470313>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.343694>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.3025699>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.2973015>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.296774>}
{'batch_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.2914217>}

CPU times: user 39.5 s, sys: 4.85 s, total: 44.4 s
Wall time: 24.8 s


In [43]:
class BatchLogs(tf.keras.callbacks.Callback):
  def __init__(self, key):
    self.key = key
    self.logs = []

  def on_train_batch_end(self, n, logs):
    self.logs.append(logs[self.key])

batch_loss = BatchLogs('batch_loss')

In [44]:
charModel.fit(dataset, epochs=20,
                     callbacks=[batch_loss])

Epoch 1/20
143/143 [==============================] - 371s 3s/step - batch_loss: 3.1252
Epoch 2/20
143/143 [==============================] - 368s 3s/step - batch_loss: 2.8651
Epoch 3/20
143/143 [==============================] - 359s 3s/step - batch_loss: 2.4808
Epoch 4/20
143/143 [==============================] - 360s 3s/step - batch_loss: 2.3918
Epoch 5/20
143/143 [==============================] - 361s 3s/step - batch_loss: 2.3353
Epoch 6/20
143/143 [==============================] - 359s 3s/step - batch_loss: 2.2868
Epoch 7/20
143/143 [==============================] - 360s 3s/step - batch_loss: 2.2362
Epoch 8/20
143/143 [==============================] - 363s 3s/step - batch_loss: 2.1919
Epoch 9/20
143/143 [==============================] - 364s 3s/step - batch_loss: 2.1516
Epoch 10/20
143/143 [==============================] - 367s 3s/step - batch_loss: 2.1073
Epoch 11/20
143/143 [==============================] - 372s 3s/step - batch_loss: 2.0763
Epoch 12/20
143/143 [=========

In [45]:
class charModel_Main(tf.Module):

  def __init__(self, encoder, decoder, input_text_processor,
               output_text_processor):
    self.encoder = encoder
    self.decoder = decoder
    self.input_text_processor = input_text_processor
    self.output_text_processor = output_text_processor

    self.output_token_string_from_index = (
        tf.keras.layers.StringLookup(
            vocabulary=output_text_processor.get_vocabulary(),
            mask_token='',
            invert=True))

    # The output should never generate padding, unknown, or start.
    index_from_string = tf.keras.layers.StringLookup(
        vocabulary=output_text_processor.get_vocabulary(), mask_token='')
    token_mask_ids = index_from_string(['', '[UNK]', '\t']).numpy()

    token_mask = np.zeros([index_from_string.vocabulary_size()], dtype=np.bool)
    token_mask[np.array(token_mask_ids)] = True
    self.token_mask = token_mask

    self.start_token = index_from_string(tf.constant('\t'))
    self.end_token = index_from_string(tf.constant('\n'))

In [46]:
charModel_trained = charModel_Main(
    encoder=charModel.encoder,
    decoder=charModel.decoder,
    input_text_processor=input_text_processor,
    output_text_processor=output_text_processor,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [47]:
def tokens_to_text(self, result_tokens):
  result_text_tokens = self.output_token_string_from_index(result_tokens)

  result_text = tf.strings.reduce_join(result_text_tokens,
                                       axis=1, separator='')

  result_text = tf.strings.strip(result_text)
  return result_text

In [48]:
charModel_Main.tokens_to_text = tokens_to_text

In [49]:
def sample(self, logits, temperature):
  # 't' is usually 1 here.

  token_mask = self.token_mask[tf.newaxis, tf.newaxis, :]

  # Set the logits for all masked tokens to -inf, so they are never chosen.
  logits = tf.where(self.token_mask, -np.inf, logits)

  if temperature == 0.0:
    new_tokens = tf.argmax(logits, axis=-1)
  else: 
    logits = tf.squeeze(logits, axis=1)
    new_tokens = tf.random.categorical(logits/temperature,
                                        num_samples=1)

  return new_tokens

In [50]:
charModel_Main.sample = sample

In [51]:
def translate_unrolled(self,
                       input_text, *,
                       max_length=50,
                       return_attention=True,
                       temperature=1.0):
  batch_size = tf.shape(input_text)[0]
  input_tokens = self.input_text_processor(input_text)
  enc_output, enc_state = self.encoder(input_tokens)

  dec_state = enc_state
  new_tokens = tf.fill([batch_size, 1], self.start_token)

  result_tokens = []
  attention = []
  done = tf.zeros([batch_size, 1], dtype=tf.bool)

  for _ in range(max_length):
    dec_input = DecoderInput(new_tokens=new_tokens,
                             enc_output=enc_output,
                             mask=(input_tokens!=0))
    
    dec_result, dec_state = self.decoder(dec_input, state=dec_state)

    attention.append(dec_result.attention_weights)

    new_tokens = self.sample(dec_result.logits, temperature)

    # If a sequence produces an `end_token`, set it `done`
    done = done | (new_tokens == self.end_token)
    # Once a sequence is done it only produces 0-padding.
    new_tokens = tf.where(done, tf.constant(0, dtype=tf.int64), new_tokens)

    # Collect the generated tokens
    result_tokens.append(new_tokens)

    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Convert the list of generates token ids to a list of strings.
  result_tokens = tf.concat(result_tokens, axis=-1)
  result_text = self.tokens_to_text(result_tokens)

  if return_attention:
    attention_stack = tf.concat(attention, axis=1)
    return {'text': result_text, 'attention': attention_stack}
  else:
    return {'text': result_text}

In [52]:
charModel_Main.translate = translate_unrolled

In [53]:
@tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
def tf_translate(self, input_text):
  return self.translate(input_text)

charModel_Main.tf_translate = tf_translate

In [67]:
%%time
input_text = tf.constant([
    've lize şunu söyleyebilirim , ajandamız dolu .', 
    've kim şeröristlerll konuşur ki ?', 
])

result = charModel_trained.tf_translate(
    input_text = input_text)

print(result['text'][0].numpy().decode())
print(result['text'][1].numpy().decode())
print()

ve çabalir heyilke istalak de bizi urlarısı olaşıy
ve bünda gerüyetin 3 '9kre kok geretteye k.ssiniye

CPU times: user 108 ms, sys: 8.78 ms, total: 117 ms
Wall time: 80.1 ms


In [74]:
example_output_tokens = tf.random.uniform(
    shape=[5, 2], minval=0, dtype=tf.int64,
    maxval=output_text_processor.vocabulary_size())
charModel_trained.tokens_to_text(example_output_tokens).numpy()

array([b'zl', b'7:', b'jf', b'\xe2\x80\x9ca', b',w'], dtype=object)

In [70]:
tf.saved_model.save(charModel_trained, 'drive/MyDrive/charBased',
                    signatures={'serving_default': charModel_trained.tf_translate})

INFO:tensorflow:Assets written to: drive/MyDrive/charBased/assets


INFO:tensorflow:Assets written to: drive/MyDrive/charBased/assets


In [ ]:
charModel_trained.save('/drive/myDrive')

In [75]:
## load test data
#"Turkish Noise Data/tr.test"
noisy_test = []
#noisy_chars = set()
normal_test = []
#normal_chars = set()
with open('/content/drive/My Drive/Turkish_Noise_Data/tr.test', "r", encoding="utf-8") as f:  
    lines = f.read().split("\n") 
    for line in lines[: min(num_samples, len(lines) - 1)]:
        #sentence = "\t" + line + "\n"  ## \t is the start char and \n is the end char
        sentence = line
        normal_test.append(sentence)
        # for char in sentence:
        #   if char not in normal_chars:
        #     normal_chars.add(char)

with open('/content/drive/My Drive/Turkish_Noise_Data/tr_noisy.test', "r", encoding="utf-8") as f:  
    lines = f.read().split("\n") 
    for line in lines[: min(num_samples, len(lines) - 1)]:
        noisy_test.append(line)
        # for char in line:
        #   if char not in noisy_chars:
        #     noisy_chars.add(char)

In [77]:
for i,sentence in enumerate(noisy_test):
  tp = 0
  fp = 0
  tn = 0
  fn = 0

  decoded_sentence = charModel_trained.tf_translate(tf.constant([sentence]))['text'][0].numpy().decode()
  print("-")
  print("Input sentence:", sentence)
  print("Decoded sentence:", decoded_sentence)
  print("Original sentence:", normal_test[i])
  
  for t,char in enumerate(normal_test[i]):
    if char == decoded_sentence[t] and sentence[t] != char:
      tp+=1
    elif char != decoded_sentence[t] and sentence[t] == char:
      fp+=1
    elif char != decoded_sentence[t] and sentence[t] != char:
      fn+=1

    if(t+1 == len(decoded_sentence)) :  ## if len is exceeded
      break

  Recall = tp/(tp+fn)
  Precision = tp/(tp+fp)
  F1 = 2*((Precision*Recall)/(Precision+Recall))
  print("Recall is: "+ str(Recall))
  print("Precision is: "+ str(Precision))
  print("F1 Score is: "+ str(F1))

-
Input sentence: bu senenin sonunda , yaklaşık bir milyar insan sosynl ağ siielerini aktif olarjk kullanıyor olacak .
Decoded sentence: bacliri atıyan abdi başım barik girzeyi nevrim kıl
Original sentence: bu senenin sonunda , yaklaşık bir milyar insan sosyal ağ sitelerini aktif olarak kullanıyor olacak .


ZeroDivisionError: ignored